In [3]:
import pytesseract
from pdf2image import convert_from_path

# Set the path to your Tesseract executable (change it if necessary)
pytesseract.pytesseract.tesseract_cmd = r'D:\Program Files\Tesseract-OCR\tesseract.exe'

# Open the PDF file
pdf_path = r'D:\Мои файлы 2019\0 всякое\doc01191220230712083423.pdf'

# Convert PDF pages to images and perform OCR
pages = convert_from_path(pdf_path, dpi=300)  # Adjust the dpi if needed

for i, page in enumerate(pages):
    try:
        # Perform OCR on the image to recognize the text
        text = pytesseract.image_to_string(page, lang='rus')

        # Output the recognized text to a text file
        output_file = f'page_{i + 1}.txt'
        with open(output_file, 'w', encoding='utf-8') as output:
            output.write(text)

        print(f'Page {i + 1} processed successfully.')

    except Exception as e:
        print(f'Error processing page {i + 1}: {str(e)}')


Page 1 processed successfully.


In [38]:
import re
import csv
import pandas as pd
from translate import Translator
from word2number import w2n


def extract_tin(text):
    tin_pattern = r"(?i)ИНН\s*(\d{10})"
    match = re.search(tin_pattern, text)
    if match:
        return match.group(1)
    else:
        return None

def extract_address(text):
    address_pattern = r"(?i)(\d{6})\s*(.*?)(\w+[\s\w]+)"
    match = re.search(address_pattern, text)
    if match:
        zip_code = match.group(1)
        street = match.group(2)
        city = match.group(3)
        return {"zip_code": zip_code, "street": street, "city": city}
    else:
        return None

def extract_patient_name(text):
    name_pattern = r"(?i)ФИО пациента:\s*(\S+\s\S+\s\S+)"
    match = re.search(name_pattern, text)
    if match:
        return match.group(1)
    else:
        return None

def extract_patient_tin(text):
    tin_pattern = r"(?i)ИНН налогоплательщика:\s*(\d{12})"
    match = re.search(tin_pattern, text)
    if match:
        return match.group(1)
    else:
        return None
    
def extract_cost(text):
    cost_pattern = r"(?i)(?:стоимость|стоимости|стоимостью):\s*(.*?)(?:\.|\n)"
    match = re.search(cost_pattern, text)
    if match:
        cost_sentence = match.group(1)
        cost_sentence_english = translate_to_english(cost_sentence)
        return cost_sentence_english
    else:
        return None

def translate_to_english(sentence):
    translator = Translator(from_lang='ru', to_lang='en')
    translation = translator.translate(sentence)
    return translation

def convert_sentence_to_number(sentence):
    words = sentence.split()
    number_words = []
    fractional_part = None
    for i, word in enumerate(words):
        if word == 'rubles':
            break
        elif word == 'kopecks':
            if i >= 3 and words[i-2].isdigit() and len(words[i-2]) == 2:
                fractional_part = words[i-2]
                number_words.append(words[i-3])  # Keep the word before "kopecks"
        try:
            number = w2n.word_to_num(word)
            number_words.append(str(number))
        except ValueError:
            number_words.append(word)
    result = " ".join(number_words)
    
    # Add the fractional part to the result if available
    if fractional_part is not None:
        result += " " + fractional_part + " kopecks"
    
    return result

def extract_digits(sentence):
    digits = []
    words = sentence.split()
    for word in words:
        if word.isdigit():
            digits.append(int(word))
    return digits

def sum_total_amount(sentence):
    words = sentence.split()
    total_amount = 0
    previous_number = 0

    for word in words:
        if word == 'rubles':
            break

        try:
            number = w2n.word_to_num(word)
            if number == 100:
                previous_number *= number
            elif number >= 1000:
                total_amount += previous_number * number
                previous_number = 0
            else:
                previous_number += number
        except ValueError:
            continue

    total_amount += previous_number
    
    
    return total_amount

# Read the text file
text_file_path = 'page_1.txt'
with open(text_file_path, 'r', encoding='utf-8') as file:
    text = file.read()

# Extract information
tin = extract_tin(text)
address = extract_address(text)
patient_name = extract_patient_name(text)
patient_tin = extract_patient_tin(text)
cost = extract_cost(text)
amount = sum_total_amount(convert_sentence_to_number(cost)) + extract_digits(cost)[0]/100


# Print the extracted information
print(f"ИНН медицинского учреждения: {tin}")
print(f"Адрес медицинского учреждения: {address}")
print(f"ФИО пациента: {patient_name}")
print(f"ИНН пациента: {patient_tin}")
print(f"Стоимость услуги: {amount}")



ИНН медицинского учреждения: 7715911798
Адрес медицинского учреждения: {'zip_code': '123007', 'street': 'г. ', 'city': 'Москва'}
ФИО пациента: Демченко Виктор Григорьевич
ИНН пациента: 773404778110
Стоимость услуги: 379300.0


In [28]:
# Create a DataFrame
data = {
    'ИНН медицинского учреждения': [tin],
    'Адрес медицинского учреждения': [address],
    'ФИО пациента': [patient_name],
    'ИНН пациента': [patient_tin],
    'Стоимость услуги': [amount]
}
df = pd.DataFrame(data)

# Display the DataFrame
df

,ИНН медицинского учреждения,Адрес медицинского учреждения,ФИО пациента,ИНН пациента,Стоимость услуги
0,7715911798,"{'zip_code': '123007', 'street': 'г. ', 'city'...",Демченко Виктор Григорьевич,773404778110,379300.0


In [7]:
import re

def extract_address(text):
    street_words = {
        'Аллея': 'ал.',
        'Бульвар': 'б-р',
        'Взвоз': 'взв.',
        'Въезд': 'взд.',
        'Дорога': 'дор.',
        'Заезд': 'ззд.',
        'Километр': 'км',
        'Кольцо': 'к-цо',
        'Коса': 'коса',
        'Линия': 'лн.',
        'Магистраль': 'мгстр.',
        'Набережная': 'наб.',
        'Переезд': 'пер-д',
        'Переулок': 'пер.',
        'Площадка': 'пл-ка',
        'Площадь': 'пл.',
        'Проезд': 'пр-д',
        'Просек': 'пр-к',
        'Просека': 'пр-ка',
        'Проселок': 'пр-лок',
        'Проспект': 'пр-кт',
        'Проулок': 'проул.',
        'Разъезд': 'рзд.',
        'Ряд(ы)': 'ряд',
        'Сквер': 'с-р',
        'Спуск': 'с-к',
        'Съезд': 'сзд.',
        'Тракт': 'тракт',
        'Тупик': 'туп.',
        'Улица': 'ул.',
        'Шоссе': 'ш.'
    }
    
    address_pattern = r"(?i)(\d{6})\s*([^\d]+)\b(" + '|'.join(street_words.keys()) + r")\b"
    match = re.search(address_pattern, text)
    if match:
        zip_code = match.group(1)
        street = match.group(2)
        street_reduction = match.group(3)
        full_street = street_words[street_reduction.lower().capitalize()]
        city_pattern = r"(?i)" + re.escape(street_reduction) + r"\s*(.*)"
        city_match = re.search(city_pattern, street)
        if city_match:
            city = city_match.group(1)
            return {"zip_code": zip_code, "street": full_street + ' ' + street, "city": city}
    
    return None

# Read the text file
text_file_path = 'page_1.txt'
with open(text_file_path, 'r', encoding='utf-8') as file:
    text = file.read()

# Extract address
address = extract_address(text)

# Print the extracted address
if address:
    print(f"ZIP Code: {address['zip_code']}")
    print(f"Street: {address['street']}")
    print(f"City: {address['city']}")
else:
    print("Address not found.")


Address not found.


In [25]:
def extract_address(sentence):
    words = sentence.split()
    address = ""
    for i, word in enumerate(words):
        if word == 'ZIP':
            address += word + " " + words[i+1]  # Include the ZIP code
        elif word in ['ул.', 'шоссе', 'просп.']:
            address += word + " " + words[i+1]  # Include the street name
        elif word == 'city':
            address += ", " + words[i+1]  # Include the city name
    return address

In [16]:
def convert_sentence_to_number(sentence):
    words = sentence.split()
    number_words = []
    fractional_part = None
    for word in words:
        if word == 'rubles':
            break
        elif word == 'kopecks':
            if fractional_part is None:
                fractional_part = word
            continue
        try:
            number = str(int(word))
            number_words.append(number)
        except ValueError:
            number_words.append(word)
    result = " ".join(number_words)

    # Add the fractional part to the result if available
    if fractional_part is not None:
        result += "." + fractional_part

    return result

# Example usage
sentence = "The cost is twenty-three thousand four hundred and fifty-six rubles 30 kopecks."
converted_sentence = convert_sentence_to_number(sentence)
print(converted_sentence)


The cost is twenty-three thousand four hundred and fifty-six


In [16]:
from word2number import w2n

def convert_sentence_to_number(sentence):
    words = sentence.split()
    number_words = []
    fractional_part = None
    for i, word in enumerate(words):
        if word == 'rubles':
            break
        elif word == 'kopecks':
            if i >= 3 and words[i-2].isdigit() and len(words[i-2]) == 2:
                fractional_part = words[i-2]
                number_words.append(words[i-3])  # Keep the word before "kopecks"
        try:
            number = w2n.word_to_num(word)
            number_words.append(str(number))
        except ValueError:
            number_words.append(word)
    result = " ".join(number_words)
    
    # Add the fractional part to the result if available
    if fractional_part is not None:
        result += " " + fractional_part + " kopecks"
    
    return result

# Example usage
sentence = "The cost is two million five hundred twenty-nine thousand two hundred eighty-four rubles and 30 kopecks."
#converted_sentence = convert_sentence_to_number(sentence)
#print(converted_sentence)

In [17]:
def extract_digits(sentence):
    digits = []
    words = sentence.split()
    for word in words:
        if word.isdigit():
            digits.append(int(word))
    return digits

# Example usage
#sentence = "The cost is five hundred twenty-nine thousand two hundred eighty-four rubles and 30 kopecks."
#digits = extract_digits(sentence)[0]
#digits/100

In [18]:
from word2number import w2n

def sum_total_amount(sentence):
    words = sentence.split()
    total_amount = 0
    previous_number = 0

    for word in words:
        if word == 'rubles':
            break

        try:
            number = w2n.word_to_num(word)
            if number == 100:
                previous_number *= number
            elif number >= 1000:
                total_amount += previous_number * number
                previous_number = 0
            else:
                previous_number += number
        except ValueError:
            continue

    total_amount += previous_number
    
    
    return total_amount

# Example usage
#sentence = "The cost is 2 1000000 5 100 29 1000 2 100 84"
#total_amount = sum_total_amount(sentence)
#print(total_amount)


In [19]:
sum_total_amount(convert_sentence_to_number(sentence)) + extract_digits(sentence)[0]/100

2529284.3

In [15]:
def extract_digits(sentence):
    digits = []
    words = sentence.split()
    for word in words:
        if word.isdigit():
            digits.append(int(word))
    if not digits:
        return None
    return digits

# Example usage
sentence = "The cost is five hundred twenty-nine thousand two hundred eighty-four rubles."
try:
    digits = extract_digits(sentence)
    print(digits)
except ValueError as e:
    print(e)

None


In [36]:
import stanza

def extract_address(text):
    nlp = stanza.Pipeline(lang='ru', processors='tokenize, pos')
    doc = nlp(text)
    
    zip_code = ""
    street = ""
    city = ""
    
    for sent in doc.sentences:
        for word in sent.words:
            if word.lemma and word.lemma.lower() == 'zip':
                if word.head != 1:  # Check if 'ZIP' is not the root word
                    zip_code = sent.words[word.head - 1].text
            elif word.upos == 'NOUN':
                if word.head == 1:  # Check if noun is a root word
                    if word.deprel == 'obl' or word.deprel == 'nmod':  # Check if noun is an oblique or a modifier
                        if street:
                            street += " " + word.text
                        else:
                            street = word.text
                elif word.deprel == 'nmod' and word.head == 3:  # Check if noun is a modifier of the city
                    city += word.text + " "
    
    if zip_code or street or city:
        return {"zip_code": zip_code, "street": street, "city": city.strip()}
    else:
        return None

# Example usage
text = "Адрес: 123456 г. Москва, ул. Ленина, дом 10"
extracted_address = extract_address(text)
print(extracted_address)


2023-07-14 08:00:05 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-07-14 08:00:05 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |

2023-07-14 08:00:05 INFO: Using device: cpu
2023-07-14 08:00:05 INFO: Loading: tokenize
2023-07-14 08:00:05 INFO: Loading: pos
2023-07-14 08:00:06 INFO: Done loading processors!


None


In [39]:
import re

def extract_address(text):
    zip_code_pattern = r"\b\d{6}\b"
    street_pattern = r"(ул\.|шоссе|просп\.)\s*\w+"
    city_pattern = r"(г\.\s*\w+)"
    
    zip_code = re.search(zip_code_pattern, text)
    street = re.search(street_pattern, text)
    city = re.search(city_pattern, text)
    
    if zip_code and street and city:
        return {
            "zip_code": zip_code.group(),
            "street": street.group(),
            "city": city.group()
        }
    else:
        return None

# Example usage
#text = "Адрес: 123456 г. Москва, ул. Ленина, дом 10"
extracted_address = extract_address(text)
print(extracted_address)

None


In [1]:
import os
from pdf2image import convert_from_path
import pytesseract

# Set the path to your Tesseract executable (change it if necessary)
pytesseract.pytesseract.tesseract_cmd = r'D:\Program Files\Tesseract-OCR\tesseract.exe'

def process_pdf_to_text(pdf_path):
    pages = convert_from_path(pdf_path, dpi=300)  # Adjust the dpi if needed
    processed_text = ""
    text_file_path = ""

    # Process only the first page
    page = pages[0]

    try:
        # Perform OCR on the image to recognize the text
        text = pytesseract.image_to_string(page, lang='rus')

        # Output the recognized text to a text file
        output_file = 'page_1.txt'
        with open(output_file, 'w', encoding='utf-8') as output:
            output.write(text)

        processed_text += "Page 1 processed successfully.\n"
        text_file_path = output_file

    except Exception as e:
        processed_text += f"Error processing page 1: {str(e)}\n"

    return processed_text, text_file_path

# Replace 'pdf_path' with the actual path to your PDF file
pdf_path = r'D:\Мои файлы 2019\0 всякое\doc01191220230712083423.pdf'

process_pdf_to_text(pdf_path)

('Page 1 processed successfully.\n', 'page_1.txt')

In [43]:
def process_uploaded_pdf(contents, filename):
    if contents is None:
        raise PreventUpdate

    _, content_string = contents.split(',')
    decoded_pdf = content_string.encode('utf-8')
    pdf_path = f'uploads/{filename}'

    with open(pdf_path, 'wb') as file:
        file.write(decoded_pdf)

    processed_text = process_pdf_to_text(pdf_path)

    text_file_path = 'page_1.txt'  # Assuming there's only one page for simplicity
    results = process_text_file(text_file_path)
    return results